In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
def compute_loglike(resid):
    sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
    ll = np.log(stats.norm.pdf(resid,loc=0,scale=sigma))
    return ll

def compute_score(yn,xn,params):
    
    xn = sm.add_constant(xn)
    resid = yn - np.matmul(xn,params)
    
    k = len(params)
    scale = (resid**2).mean()
    
    tile_resid = np.tile( resid, k)
    tile_resid = np.reshape(tile_resid, (k,xn.shape[0]) ).transpose()
    grad = tile_resid*xn/scale     
    return grad
    
def compute_hess(yn,xn,params):
    pass


def setup_shi(yn,xn,return_model=False,num_params=15):
    x1n,x2n = xn[:,0],xn[:,1:num_params]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    ll1 = compute_loglike(model1_fit.resid)
    grad1 =  compute_score(yn,x1n,model1_fit.params)
    hess1 = model1.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    ll2 = compute_loglike(model2_fit.resid)
    grad2 =  compute_score(yn,x2n,model2_fit.params)
    hess2 = model2.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2

num_params=15

# Random stuff...

In [16]:
def gen_data(beta=0,num_params=15):
    nobs = 1000
    x = np.random.normal(scale=3., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + .25*x[:,0] + .25/np.sqrt(num_params)*x[:,1:num_params].sum(axis=1) + e
    return y,x,nobs

mc_out = vuong_tests3.monte_carlo(100,gen_data,setup_shi)
vuong_tests3.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-bc & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.90 & 0.88 & 0.89 & 0.89   \\
Model 1 & 0.06 & 0.09 & 0.11 & 0.11 & 0.11   \\
Model 2 & 0.02 & 0.01 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}


In [21]:
def gen_data(beta=0):
    nobs = 500
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,10))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + .25*x[:,0] + .25/np.sqrt(9)*x[:,1:9].sum(axis=1) + e
    return y,x,nobs

mc_out = vuong_tests3.monte_carlo(100,gen_data,setup_shi)
vuong_tests3.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-bc & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.87 & 0.87 & 0.85 & 0.90   \\
Model 1 & 0.01 & 0.12 & 0.12 & 0.14 & 0.09   \\
Model 2 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}


In [22]:
print(mc_out)

(array([0.98, 0.01, 0.01]), array([0.87, 0.12, 0.01]), array([0.87, 0.12, 0.01]), array([0.85, 0.14, 0.01]), array([0.9 , 0.09, 0.01]), 5.229168579123513, 15.381521033764535, 16.90884930176897)


In [19]:
def gen_data(beta=0):
    nobs = 250
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,10))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + .25*x[:,0] + .25/np.sqrt(9)*x[:,1:9].sum(axis=1) + e
    return y,x,nobs

mc_out = vuong_tests3.monte_carlo(100,gen_data,setup_shi)
vuong_tests3.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-bc & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.89 & 0.87 & 0.90 & 0.92   \\
Model 1 & 0.03 & 0.10 & 0.13 & 0.09 & 0.08   \\
Model 2 & 0.01 & 0.01 & 0.00 & 0.01 & 0.00   \\
\hline
\end{tabular}


In [20]:
def gen_data(beta=0):
    nobs = 250
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,4))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + 1/np.sqrt(2)*x[:,3] + 1/np.sqrt(2)*x[:,2] + 1*x[:,0] + beta * x[:,1] + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,ll2,k1, grad2,hess2,k2 = setup_shi(yn,xn)


mc_out = vuong_tests3.monte_carlo(100,gen_data,setup_shi)
vuong_tests3.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-bc & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.95 & 0.94 & 0.94 & 0.97 & 0.98   \\
Model 1 & 0.01 & 0.03 & 0.02 & 0.01 & 0.01   \\
Model 2 & 0.04 & 0.03 & 0.04 & 0.02 & 0.01   \\
\hline
\end{tabular}
